# Lab 03: Document Risk Analysis

This notebook explores the document risk analysis capabilities of O-Series reasoning models by combining multiple datasets with enhanced reasoning.

It serves as a practical demonstration of fraud detection, leveraging unstructured (transaction logs) and structured (emails, transcripts, sms) data to improve decision-making.

These flagged transactions are fictitious, lab-generated credit/debit card purchases designed to illustrate common patterns of fraud (e.g., large purchases on new devices, location mismatches). In real systems, similar transaction data might flow in from bank payment networks, e‑commerce platforms, or internal databases that log user activities and device usage.

Case managers, responsible for investigating flagged transactions, often face an overwhelming number of alerts with limited context. Traditional systems generate excessive noise, making it difficult to identify truly fraudulent cases.

This notebook enhances case managers’ efficiency by:

✅ Linking transaction data with customer complaints

✅ Prioritizing high-risk cases using AI-powered reasoning

✅ Providing meaningful insights to reduce manual workload

By integrating o-series models for document risk analysis, this system ensures that case managers focus on the most critical cases, detect fraud faster, and take informed action.

In [ ]:
## This is a Test Code
"""Run this model in Python

> pip install openai
"""
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display, Markdown, HTML, Image

# Load environment variables from .env file
load_dotenv()

# To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings. 
# Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
client = OpenAI(
    base_url="https://models.inference.ai.azure.com",
    api_key=os.environ["tokenx"],
)

#Testing the model
response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "What is the capital of France?",
        }
    ],
    model="o3-mini",
    reasoning_effort="high" 
)

print(response.choices[0].message.content)

The capital of France is Paris.


### Step 1. Read the Data from two files consisting of structured and unstructured data

In [3]:
import pandas as pd
# Read the transcripts CSV file

transcripts_df = pd.read_csv('Data/transcripts_emails_data.csv')

# Read the flagged transactions file
with open('Data/flagged_transactions.txt', 'r') as file:
    flagged_transactions_df = pd.read_csv(file, delimiter='\t')

print(transcripts_df.head())
print(flagged_transactions_df.head())

  CaseID MessageType                                            Message
0   C001       Email  Subject: Duplicate Charge Alert. Body: I see t...
1   C002        Call  Transcript: Caller refused to provide SSN on t...
2   C003       Email  Subject: Basic Inquiry. Body: I’m just checkin...
3   C004        Call  Transcript: Customer disputed a $350 airfare c...
4   C005       Email  Subject: One Question. Body: Which day does my...
                       1. Customer C001 (Adams)
0                                    Rationale:
1                   - Flagged Transactions: Yes
2                 - Fraud Risk Score: 90 (High)
3     - Account Age: 0.8 years (Newish Account)
4                 - Suspicious Device Activity:


In [21]:
structured_prompt = (
    "<instructions>"
    "You are an expert in fraud detection and analysis of both structured and unstructured data. "
    "Based on the provided fraud dataset—which includes a structured file of transaction summaries and an unstructured file of call/email transcripts—perform the following tasks in an HTML format:\n"
    "1. Identify medium and high-risk cases by referencing structured attributes (such as Fraud Risk Score, Device Activity, and AML/KYC Flags).\n"
    "2. For each medium and high-risk case, analyze the unstructured transcripts (emails, calls, chats) for additional evidence of fraud or legitimate explanations.\n"
    "3. Detect any false positives by cross-referencing 'Final Note' fields or contradictory information in the transcripts. \n"
    "4. Callout false positives in a neatly formmatted table and explain the reason why was it flagged and the reason for it being false positive\n"
    "5. Summarize all findings in a clear, concise plain-text report using simple language. .\n"
    "6. Only use the information provided in the input data to generate your conclusions.\n"
    "</instructions>"
)

print(structured_prompt)


<instructions>You are an expert in fraud detection and analysis of both structured and unstructured data. Based on the provided fraud dataset—which includes a structured file of transaction summaries and an unstructured file of call/email transcripts—perform the following tasks in an HTML format:
1. Identify medium and high-risk cases by referencing structured attributes (such as Fraud Risk Score, Device Activity, and AML/KYC Flags).
2. For each medium and high-risk case, analyze the unstructured transcripts (emails, calls, chats) for additional evidence of fraud or legitimate explanations.
3. Detect any false positives by cross-referencing 'Final Note' fields or contradictory information in the transcripts. 
4. Callout false positives in a neatly formmatted table and explain the reason why was it flagged and the reason for it being false positive
5. Summarize all findings in a clear, concise plain-text report using simple language. .
6. Only use the information provided in the input d

In [22]:
messages = [{
    "role": "user",
    "content": "Extract medium and high risk transaction" + "\n" + flagged_transactions_df.to_string()
}]

response1 = client.chat.completions.create(model="gpt-4o",
                                          messages=messages)


In [23]:
display(HTML('<div style="background-color: #f0fff8; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>🔽 &nbsp; Markdown Output – Beginning</h2></hr></div>'))
display(Markdown(response1.choices[0].message.content))
display(HTML('<div style="background-color: #fff4f4; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>🔼 &nbsp; Markdown Output – End</h2></hr></div>'))

medium_high_risk = response1.choices[0].message.content


### Extracted Medium and High Risk Transactions

**High Risk Transactions (Fraud Risk Score ≥ 75):**
1. **Customer C001 (Adams)**  
   - Fraud Risk Score: 90 (High)  
   - Recent Purchases: Multiple SIM Cards, High-End Electronics  

2. **Customer C002 (Baker)**  
   - Fraud Risk Score: 95 (Very High)  
   - Recent Purchases: Old Phone, SIM Card – Brand A  

3. **Customer C006 (Franklin)**  
   - Fraud Risk Score: 88 (High)  
   - Recent Purchases: Multiple Gift Cards  

4. **Customer C011 (Keller)**  
   - Fraud Risk Score: 90 (High)  
   - Recent Purchases: High-End Watches  

5. **Customer C015 (Owens)**  
   - Fraud Risk Score: 75 (High)  
   - Recent Purchases: Foreign Currency  

6. **Customer C017 (Quinn)**  
   - Fraud Risk Score: 82 (High)  
   - Recent Purchases: Cryptocurrency  

7. **Customer C020 (Taylor)**  
   - Fraud Risk Score: 92 (High)  
   - Recent Purchases: Plane Tickets, Luxury Hotel  

8. **Customer C024 (Xie)**  
   - Fraud Risk Score: 77 (High)  
   - Recent Purchases: Luxury Shoes  

9. **Customer C026 (Zhang)**  
   - Fraud Risk Score: 85 (High)  
   - Recent Purchases: Electronics, Crypto  

10. **Customer C029 (Chan)**  
    - Fraud Risk Score: 93 (Very High)  
    - Recent Purchases: High-Value Electronics  

---

**Medium Risk Transactions (Fraud Risk Score 50–74):**
1. **Customer C004 (Davis)**  
   - Fraud Risk Score: 65 (Medium)  
   - Recent Purchases: Airline Tickets  

2. **Customer C007 (Garcia)**  
   - Fraud Risk Score: 58 (Medium)  
   - Recent Purchases: Jewelry  

3. **Customer C009 (Ingram)**  
   - Fraud Risk Score: 72 (Medium)  
   - Recent Purchases: Electronics Return  

4. **Customer C013 (Miller)**  
   - Fraud Risk Score: 53 (Medium)  
   - Recent Purchases: Car Parts  

5. **Customer C019 (Sanchez)**  
   - Fraud Risk Score: 60 (Medium)  
   - Recent Purchases: Mobile Accessories  

6. **Customer C022 (Vargas)**  
   - Fraud Risk Score: 70 (Medium)  
   - Recent Purchases: Bulk Gift Cards  

7. **Customer C028 (Braun)**  
   - Fraud Risk Score: 68 (Medium)  
   - Recent Purchases: Designer Bag  

In [35]:
messages = [{
    "role": "user",
    "content": structured_prompt + "\n" + medium_high_risk + "\n" + transcripts_df.to_string()
}]

response = client.chat.completions.create(model="o1", 
                                          messages=messages)


In [ ]:
display(HTML('<div style="background-color: #f0fff8; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>🔽 &nbsp; Markdown Output – Beginning</h2></hr></div>'))
display(Markdown(response.choices[0].message.content))
display(HTML('<div style="background-color: #fff4f4; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>🔼 &nbsp; Markdown Output – End</h2></hr></div>'))

<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>Fraud Analysis Report</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 20px;
            padding: 0;
        }
        h1, h2, h3 {
            color: #333;
        }
        .risk-section {
            margin-bottom: 30px;
        }
        .false-positive-table, .false-positive-table th, .false-positive-table td {
            border: 1px solid #ccc;
            border-collapse: collapse;
        }
        .false-positive-table th, .false-positive-table td {
            padding: 8px;
            text-align: left;
        }
        .report {
            background-color: #f9f9f9;
            border: 1px solid #ccc;
            padding: 15px;
            max-width: 600px;
            margin-top: 30px;
        }
    </style>
</head>
<body>

<h1>Fraud Analysis Report</h1>

<!-- 1. Identify Medium and High-Risk Cases -->
<div class="risk-section">
    <h2>1. Medium and High-Risk Cases</h2>
    <h3>High-Risk (Fraud Risk Score ≥ 75)</h3>
    <ul>
        <li><strong>C001 (Adams)</strong> – Score: 90, Recent Purchases: Multiple SIM Cards, High-End Electronics</li>
        <li><strong>C002 (Baker)</strong> – Score: 95, Recent Purchases: Old Phone, SIM Card – Brand A</li>
        <li><strong>C006 (Franklin)</strong> – Score: 88, Recent Purchases: Multiple Gift Cards</li>
        <li><strong>C011 (Keller)</strong> – Score: 90, Recent Purchases: High-End Watches</li>
        <li><strong>C015 (Owens)</strong> – Score: 75, Recent Purchases: Foreign Currency</li>
        <li><strong>C017 (Quinn)</strong> – Score: 82, Recent Purchases: Cryptocurrency</li>
        <li><strong>C020 (Taylor)</strong> – Score: 92, Recent Purchases: Plane Tickets, Luxury Hotel</li>
        <li><strong>C024 (Xie)</strong> – Score: 77, Recent Purchases: Luxury Shoes</li>
        <li><strong>C026 (Zhang)</strong> – Score: 85, Recent Purchases: Electronics, Crypto</li>
        <li><strong>C029 (Chan)</strong> – Score: 93, Recent Purchases: High-Value Electronics</li>
    </ul>

    <h3>Medium-Risk (Fraud Risk Score 50–74)</h3>
    <ul>
        <li><strong>C004 (Davis)</strong> – Score: 65, Recent Purchases: Airline Tickets</li>
        <li><strong>C007 (Garcia)</strong> – Score: 58, Recent Purchases: Jewelry</li>
        <li><strong>C009 (Ingram)</strong> – Score: 72, Recent Purchases: Electronics Return</li>
        <li><strong>C013 (Miller)</strong> – Score: 53, Recent Purchases: Car Parts</li>
        <li><strong>C019 (Sanchez)</strong> – Score: 60, Recent Purchases: Mobile Accessories</li>
        <li><strong>C022 (Vargas)</strong> – Score: 70, Recent Purchases: Bulk Gift Cards</li>
        <li><strong>C028 (Braun)</strong> – Score: 68, Recent Purchases: Designer Bag</li>
    </ul>
</div>

<!-- 2. Analysis of Unstructured Transcripts for Each Medium and High-Risk Case -->
<div class="risk-section">
    <h2>2. Analysis of Unstructured Transcripts</h2>

    <h3>High-Risk Cases</h3>
    <ul>
        <li>
            <strong>C001 (Adams):</strong> Email about a duplicate $499.99 charge. Customer claims they only made one purchase. No direct contradictions; requests investigation.
        </li>
        <li>
            <strong>C002 (Baker):</strong> Call where the caller refused to provide SSN, demanded reversal of unauthorized charges. Heightened suspicion due to refusal of ID checks.
        </li>
        <li>
            <strong>C006 (Franklin):</strong> Call transcript shows caller tried skipping identity checks, requested urgent refund of $200 without proof. Indicates possible fraud.
        </li>
        <li>
            <strong>C011 (Keller):</strong> Call with aggressive refusal to answer security questions, threatened legal action if $1,000+ charges not reversed. Remains suspicious.
        </li>
        <li>
            <strong>C015 (Owens):</strong> SMS about a $500 foreign currency transaction. Asking to confirm legitimacy. No contradictory statements, but still a high-risk trigger.
        </li>
        <li>
            <strong>C017 (Quinn):</strong> Call with uncertain details about a $900 cryptocurrency purchase; caller gave contradictory info on location/date. Suggests heightened risk.
        </li>
        <li>
            <strong>C020 (Taylor):</strong> Email from overseas location acknowledging $600 hotel and flight charges. Claims they are legitimate, but flagged as high risk due to travel.
        </li>
        <li>
            <strong>C024 (Xie):</strong> Email stating $450 luxury shoe purchase is valid (for work travel). Remains high risk given purchase profile; no proof or contradiction provided.
        </li>
        <li>
            <strong>C026 (Zhang):</strong> SMS about a $750 laptop purchase flagged due to new shipping address. Customer insists it is legitimate but remains under high-risk scrutiny.
        </li>
        <li>
            <strong>C029 (Chan):</strong> Chat ended abruptly after customer refused to verify $1,500 electronics purchase. Location mismatch. Strongly suspicious behavior.
        </li>
    </ul>

    <h3>Medium-Risk Cases</h3>
    <ul>
        <li>
            <strong>C004 (Davis):</strong> Call about a $350 airfare dispute. Customer later admitted it was legitimate for an upcoming trip, suggesting initial dispute was confusion.
        </li>
        <li>
            <strong>C007 (Garcia):</strong> Email alleging ID theft for a $2,000 jewelry purchase in another state. Denies authorizing the transaction; no contradictory info found.
        </li>
        <li>
            <strong>C009 (Ingram):</strong> Chat denying travel out of state for a transaction in Florida. Customer denies any such trip. Could be fraudulent or mistaken identity.
        </li>
        <li>
            <strong>C013 (Miller):</strong> Call uncertain about a $120 car part brand. The customer recalls ordering “something” but details are fuzzy. Moderate suspicion remains.
        </li>
        <li>
            <strong>C019 (Sanchez):</strong> Call where customer is confused about $50 accessory charges. Possibly a family member used the card. Unclear but still flagged medium risk.
        </li>
        <li>
            <strong>C022 (Vargas):</strong> Email inquiring about purchasing ten $100 gift cards in one transaction. Unusual bulk purchase. Remains under medium-risk investigation.
        </li>
        <li>
            <strong>C028 (Braun):</strong> Call initially claimed identity theft on a $400 purchase, then changed the story saying a friend borrowed the card. Contradictory explanation.
        </li>
    </ul>
</div>

<!-- 3. & 4. Detecting False Positives and Table -->
<div class="risk-section">
    <h2>3. False Positives Analysis</h2>
    <p>
        After examining transcripts for contradictions or legitimate explanations, only one case appears to be a clear false positive:
    </p>

    <table class="false-positive-table">
        <thead>
            <tr>
                <th>Customer ID</th>
                <th>Reason Flagged as Suspicious</th>
                <th>Reason for False Positive</th>
            </tr>
        </thead>
        <tbody>
            <tr>
                <td>C004 (Davis)</td>
                <td>$350 airfare charge dispute; transaction triggered a medium-risk alert.</td>
                <td>Customer later admitted to booking the flight for next month. The confusion over posting date caused the initial dispute, which turned out to be a legitimate purchase.</td>
            </tr>
        </tbody>
    </table>
</div>

<!-- 5. Plain-Text Summary -->
<div class="report">
    <h3>Plain-Text Summary of Findings</h3>
    <p>
        After reviewing the transactions and associated transcripts, the following observations were made:
    </p>
    <p>
        • High-risk customers generally showed patterns such as refusing standard security checks, insisting on immediate refunds without proof, or showing contradictory travel or purchase details.  
        • Medium-risk customers raised concerns due to unusual purchases, partial recall of transactions, or purchases typical of fraud patterns (e.g., bulk gift cards).  
        • One notable false positive was Customer C004, who initially disputed airfare charges but later acknowledged the purchase. This confusion was misinterpreted as fraud.  
        • No other medium or high-risk cases showed clear contradictions or final notes indicating a legitimate explanation for the flagged risk.  
        • Thus, only one clear false positive (C004) was identified. All other cases remain under suspicion or require further investigation.
    </p>
</div>

</body>
</html>

In [ ]:
html_content_o1 = response.choices[0].message.content

# Define the file path where you want to save the HTML content
file_path = 'Output/fraud_analysis_o1.html'

# Write the HTML content to the file
with open(file_path, 'w', encoding='utf-8') as file:
    file.write(html_content_o1)

print(f"HTML content has been written to {file_path}")

HTML content has been written to fraud_analysis.html


## Using GPT-4o for Fraud Analysis


In [ ]:
response_gpt4o = client.chat.completions.create(model="gpt-4o", 
                                          messages=messages)

html_content_gpt4o = response_gpt4o.choices[0].message.content

# Define the file path where you want to save the HTML content
file_path = 'Output/fraud_analysis_gpt4o.html'

# Write the HTML content to the file
with open(file_path, 'w', encoding='utf-8') as file:
    file.write(html_content_gpt4o)

print(f"HTML content has been written to {file_path}")


HTML content has been written to fraud_analysis_gpt4o.html


## Conclusion

In scenarios involving thousands of alerts, o3-mini models can be leveraged for initial bulk data analysis to narrow down the alerts to a manageable subset. Subsequently, more advanced models like o1 can be utilized to scrutinize these alerts, enabling a more cost-efficient and accurate workflow. This layered approach optimizes resource allocation and enhances the precision of fraud detection.